In [ ]:
!pip install -q kaggle

In [ ]:
#upload API kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"srikresna","key":"c90f9c19af6b129911ae9410dd23cde4"}'}

In [ ]:
#buat direktori kaggle
!mkdir ~/.kaggle

In [ ]:
#pindahkan API kaggle ke dalam direktori yang telah dibuat
!cp kaggle.json ~/.kaggle/

In [ ]:
#berikan permission kepada direktori 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#copy API dataset
!kaggle datasets download -d fredericobreno/play-tennis

play-tennis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#unzip folder
!unzip play-tennis.zip

Archive:  play-tennis.zip
  inflating: play_tennis.csv         


In [ ]:
import pandas as pd

#baca file playtennis
df = pd.read_csv('play_tennis.csv')

In [ ]:
#cari informasi terlebih dahulu
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   day       14 non-null     object
 1   outlook   14 non-null     object
 2   temp      14 non-null     object
 3   humidity  14 non-null     object
 4   wind      14 non-null     object
 5   play      14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


In [ ]:
#karena kolom day tidak diperlukan dalam klasifikasi decision tree, maka bisa kita hapus

df.drop('day', axis = 1, inplace = True)

#cek apakah sudah menghilang
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   outlook   14 non-null     object
 1   temp      14 non-null     object
 2   humidity  14 non-null     object
 3   wind      14 non-null     object
 4   play      14 non-null     object
dtypes: object(5)
memory usage: 688.0+ bytes


In [ ]:
#karena data yang ada adalah kategorikal, lakukan one-hot encoding
df['outlook'] = pd.factorize(df.outlook)[0]
df['temp'] = pd.factorize(df.temp)[0]
df['humidity'] = pd.factorize(df.humidity)[0]
df['wind'] = pd.factorize(df.wind)[0]

df

,outlook,temp,humidity,wind,play
0,0,0,0,0,No
1,0,0,0,1,No
2,1,0,0,0,Yes
3,2,1,0,0,Yes
4,2,2,1,0,Yes
5,2,2,1,1,No
6,1,2,1,1,Yes
7,0,1,0,0,No
8,0,2,1,0,Yes
9,2,1,1,0,Yes


In [ ]:
#pisahkan atribut dan label
x = df[['outlook', 'temp', 'humidity', 'wind']]
y = df['play']

#bagi menjadi data latih & data uji
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=123) 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

#membuat model decision tree
tree_model = DecisionTreeClassifier()

#melatih model dengan menggunakan data latih
tree_model = tree_model.fit(x_train, y_train)

In [ ]:
#evaluasi model
from sklearn.metrics import accuracy_score

y_pred = tree_model.predict(x_test)
acc_score = round(accuracy_score(y_pred, y_test), 3)
print('Accuracy : ', acc_score)

Accuracy :  1.0


In [ ]:
#contoh prediksi decision tree kali ini, ([[outlook, temp,	humidity,	wind]])
#anda bisa mengedit 4 digit angka dalam kolom ini
prediksi = tree_model.predict([[2, 0, 1, 0]])[0]

if prediksi == 'Yes':
  print('BERMAIN')
elif prediksi == 'No':
  print('TIDAK BERMAIN')


BERMAIN


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
#visualisasi decision tree menggunakan graphviz dari sklearn

from sklearn.tree import export_graphviz
export_graphviz(
    tree_model,
    out_file = "playtennis_tree.dot",
    feature_names = ['outlook','temp','humidity','wind'],
                       class_names = 'play',
                       filled=True)

#anda akan mendapat file .dot, anda bisa download untuk melihat grap decision tree nya



In [ ]:
from graphviz import Source
path = '/content/playtennis_tree.dot'
s = Source.from_file(path)
s.view()


'/content/playtennis_tree.dot.pdf'